<a href="https://colab.research.google.com/github/olaviinha/SloppyButchery/blob/main/SloppyUpsampler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Sloppy Upsampler <font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">Audio upsampler</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/SloppyButchery" target="_blank"><font color="#999" size="4">Github</font></a>

Poor man's upsampling by traditional methods: 
- convert to 44.1 kHz with a decent interpolation filter (ffmpeg -dither_scale 1 -resampler 'swr' -filter_size 32 -phase_shift 10)
- pitch up one octave
- highpass
- mix on original with low volume

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.<br>
#@markdown <small>Mounting Drive will enable this notebook to save outputs directly to your Drive. Otherwise you will need to copy/download them manually from this notebook.</small>

force_setup = False
repositories = []
pip_packages = ''
apt_packages = 'ffmpeg'
mount_drive = True #@param {type:"boolean"}
skip_setup = False #@ param {type:"boolean"}

# Download the repo from Github
import os
from google.colab import output
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !pip -q install import-ipynb {pip_packages}
  if apt_packages != '':
    !apt-get update && apt-get install {apt_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if mount_drive == True:
  if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
    drive_root = '/content/drive/My Drive'
  if not os.path.isdir('/content/mydrive'):
    os.symlink('/content/drive/My Drive', '/content/mydrive')
    drive_root = '/content/mydrive/'
  drive_root_set = True
else:
  create_dirs(['/content/faux_drive'])
  drive_root = '/content/faux_drive/'

if len(repositories) > 0 and skip_setup == False:
  for repo in repositories:
    %cd /content/
    install_dir = fix_path('/content/'+path_leaf(repo).replace('.git', ''))
    repo = repo if '.git' in repo else repo+'.git'
    !git clone {repo}
    if os.path.isfile(install_dir+'setup.py') or os.path.isfile(install_dir+'setup.cfg'):
      !pip install -e ./{install_dir}
    if os.path.isfile(install_dir+'requirements.txt'):
      !pip install -r {install_dir}/requirements.txt

if len(repositories) == 1:
  %cd {install_dir}

dir_tmp = '/content/tmp/'
create_dirs([dir_tmp])

import time, sys
from datetime import timedelta
import math


import numpy as np
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt

def highpass_audio(input, cutoff=10000, fs=44100, order=6):
  if type(input) == np.ndarray:
    data = input
    sr = fs
  else:
    data, sr = librosa.load(input, sr=None, mono=False)
  b, a = butter(order, cutoff, fs=fs, btype='highpass', analog=False)
  y = lfilter(b, a, data)
  return y

def apply_highpass_mix(input, hipass_vol=0.5, mix_vol=0.8, cutoff=10000, fx=44100, order=6):
  if type(input) == np.ndarray:
    data = input
    sr = fs
  else:
    data, sr = librosa.load(input, sr=None, mono=False)
  octave_up = librosa.effects.pitch_shift(data, sr=sr, n_steps=12) * hipass_vol
  hipass = highpass_audio(octave_up, cutoff, fx, order)
  return data*mix_vol+hipass

output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.', time=True)

In [ ]:
#@title # Upsample
input = "" #@param {type:"string"}
output_dir = "" #@param {type:"string"}
hipass_vol = 0.5 #@ param {type: "number"}
cutoff = 10000 #@ param {type: "number"}
mix_vol = 0.8 #@ param {type: "number"}
#@markdown <small>Unchecking `show_players` is recommended if you process a lot of files in one go.</small>
show_players = True #@param {type: "boolean"}

new_sr = 44100
new_format = 'pcm_s24le'

end_session_when_done = False #@ param {type: "boolean"}

uniq_id = gen_id()

if os.path.isfile(drive_root+input):
  inputs = [drive_root+input]
  dir_in = path_dir(drive_root+input)
elif input != '' and os.path.isdir(drive_root+input):
  dir_in = drive_root+fix_path(input)
  # What to do if input is directory path
  inputs = list_audio(dir_in)
elif os.path.isdir(drive_root+input) and '*' in input:
  dir_in = path_dir(drive_root+input)
  inputs = glob(drive_root+input)
else:
  op(c.fail, 'FAIL!', 'Input should be a path to a file or a directory.')
  sys.exit('Input not understood.')

# Output
if output_dir == '':
  dir_out = dir_in
else:
  if not os.path.isdir(drive_root+output_dir):
    os.mkdir(drive_root+output_dir)
  dir_out = drive_root+fix_path(output_dir)
  
timer_start = time.time()
total = len(inputs)

# -- DO THINGS --
for i, input in enumerate(inputs, 1):

  if i % 10: output.clear()
  
  ndx_info = str(i)+'/'+str(total)+' '
  op(c.title, ndx_info+'Processing', input.replace(drive_root, ''), time=True)

  file_tmp = dir_tmp+basename(input)+'.wav'
  file_out = dir_out+uniq_id+'_44khz_'+basename(input)+'.wav'

  if show_players == True: 
    op(c.okb, 'Original audio')
    audio_player(input)

  !ffmpeg -hide_banner -loglevel panic -i "{input}" -acodec {new_format} -ar {new_sr} "{file_tmp}"
  
  hpf = apply_highpass_mix(file_tmp)
  hpf = apply_highpass_mix(file_tmp, hipass_vol=hipass_vol, mix_vol=mix_vol, cutoff=cutoff, fx=new_sr)

  if show_players == True: 
    op(c.okb, 'Upsampled audio')
    audio_player(hpf)

  sf.write(file_out, hpf.T, new_sr, subtype='PCM_24')
  if os.path.isfile(file_out):
    op(c.ok, 'Upsampled audio saved as', file_out.replace(drive_root, ''))
  else:
    op(c.fail, 'Error saving file', file_out)
  print()
# -- END THINGS --

timer_end = time.time()

print()
op(c.okb, 'Elapsed', timedelta(seconds=timer_end-timer_start), time=True)
op(c.ok, 'FIN.')

if end_session_when_done is True: end_session()